# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [15]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = './data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [16]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df

5 object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S
...,...,...,...,...,...
413,"Spector, Mr. Woolf",male,A.5. 3236,None,S
414,"Oliva y Ocana, Dona. Fermina",female,PC 17758,C105,C
415,"Saether, Mr. Simon Sivertsen",male,SOTON/O.Q. 3101262,None,S
416,"Ware, Mr. Frederick",male,359309,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [17]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.7800138095537004
time : 0.0868229866027832 sec


In [18]:
# "均值編碼" + 邏輯斯迴歸
data = pd.concat([df[:train_num], train_Y], axis=1)
for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    # 重要: groupby產出為 pd.Series(一維陣列), 需透過.reset_index(): 加上index, 轉為 pd.DataFrame(二維陣列) !!
    #print(mean_df)
    mean_df.columns = [c, f'{c}_mean']
    #print(mean_df)
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
display(data)
data = data.drop(['Survived'] , axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

,Survived,Name_mean,Sex_mean,Ticket_mean,Cabin_mean,Embarked_mean
0,0,0,0.188908,0.0,0.299854,0.336957
1,1,1,0.742038,1.0,1.000000,0.553571
2,1,1,0.742038,1.0,0.299854,0.336957
3,1,1,0.742038,0.5,0.500000,0.336957
4,0,0,0.188908,0.0,0.299854,0.336957
...,...,...,...,...,...,...
886,0,0,0.188908,0.0,0.299854,0.336957
887,1,1,0.742038,1.0,1.000000,0.336957
888,0,0,0.742038,0.0,0.299854,0.336957
889,1,1,0.188908,1.0,1.000000,0.553571


shape : (891, 5)
score : 1.0
time : 0.03794360160827637 sec


In [ ]:
# 結果: 以上鐵達尼生存預測範例, 均值編碼與標籤編碼兩者比較:均值編碼效果過好 (正確率會到達 1.0)，會overfit !!

In [ ]:
'''
但這主要是因為'Name'與'Ticket'兩個欄位的種類數眾多
如果均值編碼時加入這兩個欄位的均值會過擬合( overfitting )
因此下列程式碼中，需要先移除這兩個欄位，再做均值編碼
'''

In [19]:
# 如果欄位類別數太多, 需要先剔除, 否則會 overfit (正確率會到達 1.0):　先剔除'Name'與'Ticket'
train_X.nunique()

Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
dtype: int64

In [22]:
# "均值編碼" + 邏輯斯迴歸

df_d = df.drop(['Name','Ticket'], axis=1)
data = pd.concat([df_d[:train_num], train_Y], axis=1)

for c in df_d.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    # 重要: groupby產出為 pd.Series(一維陣列), 需透過.reset_index(): 加上index, 轉為 pd.DataFrame(二維陣列) !!
    #print(mean_df)
    mean_df.columns = [c, f'{c}_mean']
    #print(mean_df)
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
display(data)
data = data.drop(['Survived'] , axis=1)
estimator = LogisticRegression()
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

,Survived,Sex_mean,Cabin_mean,Embarked_mean
0,0,0.188908,0.299854,0.336957
1,1,0.742038,1.000000,0.553571
2,1,0.742038,0.299854,0.336957
3,1,0.742038,0.500000,0.336957
4,0,0.188908,0.299854,0.336957
...,...,...,...,...
886,0,0.188908,0.299854,0.336957
887,1,0.742038,1.000000,0.336957
888,0,0.742038,0.299854,0.336957
889,1,0.188908,1.000000,0.553571


shape : (891, 5)
score : 0.835019772770071
time : 0.03495335578918457 sec
